In [1]:
#export
from IPython.display import Latex
from pytex.helpers import varArgFunc

In [2]:
"NOTE: need to add eager execution, it will definitely help"

'NOTE: need to add eager execution, it will definitely help'

In [3]:
#export
from pytex.helpers import *
class Var():
    def __init__(self, name, latex=None):
        self._name = str(name)
        self._fn, self._latex = None, None
        self._isTopLevel = True
    
    @property
    def name(self): return self._name
    @name.setter
    def name(self, n): self._name = n
    def setName(self, n): 
        self._name = n
        return self
    
    @property
    def backward_function(self): return self._fn
    @backward_function.setter
    def backward_function(self, fn): self._fn = fn
      
    def __str__(self): return self._name
    def __repr__(self): return self.__str__()
    
    def apply(self, f, *args):
        n = Var('temp')
        a = makeVar(*args)
        n.backward_function = f(*a) if not isinstance(a,Var) else f(a)
        return n
    
    def __add__(self, other):
        return self.apply(BinaryFunc(ADD), self, other)
    def __sub__(self, other):
        return self.apply(BinaryFunc(SUB), self, other)
    def __mul__(self, other):
        return self.apply(BinaryFunc(MUL), self, other)
    def __pow__(self, other):
        return self.apply(BinaryFunc(POW), self, other)
    def __or__(self, other):
        return self.apply(BinaryFunc(OR), self, other)
    
    def __lt__(self, other):
        return self.lt(other)
    def __gt__(self, other):
        return self.gt(other)
    def __le__(self, other):
        return self.le(other)
    def __ge__(self, other):
        return self.ge(other)
    def __eq__(self, other):
        return self.eq(other)
    def __ne__(self, other):
        return self.ne(other)
    
    def equals(self, other):
        return self.apply(BinaryFunc(EQ), self, other)
    def not_equals(self, other):
        return self.apply(BinaryFunc(NE), self, other)
    def eq(self,other):
        return self.equals(other)
    def ne(self,other):
        return self.not_equals(other)
    def lt(self, other):
        return self.apply(BinaryFunc(LT), self, other)
    def gt(self, other):
        return self.apply(BinaryFunc(GT), self, other)
    def le(self, other):
        return self.apply(BinaryFunc(LE), self, other)
    def ge(self, other):
        return self.apply(BinaryFunc(GE), self, other)
    
    def __neg__(self):
        return self.apply(NegateFunc(), self)
    def root(self, other):
        return self.apply(RootFunc(), self, other)
    def __truediv__(self, other):
        return self.apply(FracFunc(), self, other)
    
    def build(self):
        if self._latex is not None: return self._latex
        else:
            self._latex = self.name if self._fn is None else self._fn.build()
            return self._latex
        
    def __call__(self, *others):
        return self.apply(FuncApplication(self), *others)

def isVar(a):
    return isinstance(a, Var)

def makeVar(*args):
    return varArgFunc(lambda arg: arg if isVar(arg) else Var(str(arg)), *args)

def addEnv(s, env='gather'):
    env = '{' + env + '}'
    return r'\begin' + env + str(s) + r'\end' + env


In [89]:
#export
class BasicFunc():
    def __init__(self, operator):
        self._operator = operator
        
    @property
    def precedence(self):
        return self.operator.precedence
    
    @property
    def operator(self):
        return self._operator
    
    @property
    def mid(self):
        return self._operator.mid
    
    # can only be called with args of type Var
    def __call__(self, *args):
        raise NotImplementedError()
        
    def get_surrounded_with_parens(self, string):
        return '(' + string + ')'
    
    def should_surround_with_parens(self, var_instance):
        return (var_instance.backward_function is not None and 
                var_instance.backward_function.precedence < self.precedence)

In [90]:
#export
class BinaryFunc(BasicFunc):
    def __init__(self, operator):
        super().__init__(operator)
        self._a1, self._a2 = None, None
        
    def build(self):
        a1, a2 = self.get_built_strings()
        return f'{a1}{self.mid}{a2}'

    def __call__(self, a1, a2):
        self._a1, self._a2 = a1, a2
        return self
    
    def get_built_strings(self):
        if self._a1 is None or self._a2 is None:
            raise Exception('args not passed to function. need to call the class')
        a1, a2 = self._a1.build(), self._a2.build()
        if self.should_surround_with_parens(self._a1):
            a1 = self.get_surrounded_with_parens(a1)
        if self.should_surround_with_parens(self._a2):
            a2 = self.get_surrounded_with_parens(a2)
        return a1, a2

In [99]:
#export
class FracFunc(BasicFunc):
    def __init__(self):
        super().__init__(DIV)
        self.num, self.den = None, None
        
    def __call__(self, num, den):
        self.num, self.den = num, den
        return self
    
    def build(self):
        return (r"\frac{" + 
                self.num.build() + r"}{" + self.den.build() + r"}")
    
class RootFunc(BasicFunc):
    def __init__(self):
        super().__init__(MAX)
        self.num, self.root = None, None
    def __call__(self, num, root):
        self.num, self.root = num, root
        return self
    def build(self):
        return(r"\sqrt[\leftroot{-1}\uproot{1}" + self.root.build() 
               + r"]{" + self.num.build() + r"}")

class NegateFunc(BasicFunc):
    def __init__(self):
        super().__init__(MAX)
        self.value = None
    def __call__(self, value):
        self.value = value
        return self
    def build(self):
        return f'-{self.value.build()}'
    
class IntegralFunc(BasicFunc):
    def __init__(self):
        super().__init__(MAX)
        self.lower, self.upper, self.func = [None]*3
    def __call__(self, lower, upper, func):
        self.lower, self.upper, self.func = lower, upper, func
        return self
    def build(self):
        pass
    
class FuncApplication(BasicFunc):
    def __init__(self, f):
        super().__init__(MAX)
        self.f, self.args = f, None
        
    def __call__(self, *args):
        if len(args) == 0: self.args = None
        elif isinstance(args, tuple): self.args = list(args)
        else: raise Exception('invalid type passed')
        return self
        
    def build(self):
        if self.args is None: return self.getNoArgsFunc()
        else: return self.getFuncWithArgs()
    
    def getNoArgsFunc(self):
        return self.getBuiltF() + '()'
    
    def getFuncWithArgs(self):
        return self.getBuiltF() + self.getArgsForBuilding()
    
    def getArgsForBuilding(self):
        s = '('
        for arg in self.args:
            s += arg.build() + ','
        s = s[:-1]
        return s + ')'
    
    def getBuiltF(self):
        # need to pass argument
        return self.f.build()

In [100]:
from pytex.platforms.jupyter import latex, platex
a,b,c,d = makeVar('a','b','c','d')
k = platex(a.le(-b))
l = platex(c.not_equals(d))

\begin{gather*}a \leq -b\end{gather*}
\begin{gather*}c \neq d\end{gather*}


In [102]:
latex(a/b)

<IPython.core.display.Latex object>

In [43]:
latex((a+b+c).equals(1)(1))

<IPython.core.display.Latex object>

In [32]:
a,b,c,d = makeVar('a','b','c','d')
latex(c.ne((a+a-b)*(a**2) | (b+b)**2).ne(c))

<IPython.core.display.Latex object>

In [103]:
!./notebook2script.py ./Variable.ipynb

Converted /home/default/Documents/py-latex/pytex/notebooks/Variable.ipynb to /home/default/Documents/py-latex/pytex/pytex/Variable.py
